In [19]:
# Set display options to show the full output without truncation
import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column width
pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame from being split across lines


def format_brl(value: float) -> str:
    """
    Converts a float number into Brazilian Real (BRL) format.
    
    Args:
        value (float): The float number to be converted.
        
    Returns:
        str: The formatted string in BRL (e.g., 'R$ 14.547,45').
    """
    return f"R$ {value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")



def numero_ordinal(n):
    unidades = ["", "primeiro", "segundo", "terceiro", "quarto", "quinto", "sexto", "sétimo", "oitavo", "nono"]
    dezenas = ["", "décimo", "vigésimo", "trigésimo", "quadragésimo", "quinquagésimo", "sexagésimo", "septuagésimo", "octogésimo", "nonagésimo"]
    centenas = ["", "centésimo", "ducentésimo", "trecentésimo", "quadringentésimo", "quingentésimo", "sexcentésimo", "septingentésimo", "octingentésimo", "nongentésimo"]
    milhares = ["milésimo"]  # Para milhares, mantemos apenas o sufixo

    def ordinal_de_unidade(n):
        return unidades[n]

    def ordinal_de_dezena(n):
        dezena = n // 10
        unidade = n % 10
        if unidade == 0:
            return dezenas[dezena]
        else:
            return dezenas[dezena] + " " + unidades[unidade]

    def ordinal_de_centena(n):
        centena = n // 100
        resto = n % 100
        if resto == 0:
            return centenas[centena]
        else:
            if resto < 10:
                return centenas[centena] + " " + ordinal_de_unidade(resto)
            else:
                return centenas[centena] + " " + ordinal_de_dezena(resto)

    def ordinal_de_milhar(n):
        milhar = n // 1000
        resto = n % 1000
        if milhar == 1:
            prefixo = "milésimo"
        else:
            prefixo = ordinal_de_unidade(milhar) + " milésimo"
        
        if resto == 0:
            return prefixo
        elif resto < 100:
            return prefixo + " " + ordinal_de_dezena(resto)
        else:
            return prefixo + " " + ordinal_de_centena(resto)

    if n < 10:
        return ordinal_de_unidade(n)
    elif n < 100:
        return ordinal_de_dezena(n)
    elif n < 1000:
        return ordinal_de_centena(n)
    elif n < 10000:
        return ordinal_de_milhar(n)
    else:
        return "Número muito grande para ser representado automaticamente"


def Numero_ordinal(n):
    return numero_ordinal(n)[0].upper() + numero_ordinal(n)[1:] 

def clean_currency(value):
    try:
        return float(value.replace('R$', '').replace('.', '').replace(',', '.').strip())
    except Exception as e:
        return 0

In [20]:
df = pd.read_csv('SALARIOS_SERVIDORES_UNB_AGOSTO.csv')

UNB = df.copy()
    
def limpar_rem(datafr):
    df = datafr
    df['REMUNERACAO BRUTA'] = df['REMUNERACAO BRUTA'].apply(clean_currency)
    df['REMUNERACAO LIQUIDA'] = df['REMUNERACAO LIQUIDA'].apply(clean_currency)
limpar_rem(UNB)
UNB.drop_duplicates(inplace=True)
UNB_sorted = UNB.sort_values(by='REMUNERACAO BRUTA', ascending=False).reset_index(drop=True)
# # UNB_sorted.head()
# # UNB_sorted.to_csv('UNB_RANKING.csv', index = False)
UNB_sorted['Ranking'] = UNB_sorted['REMUNERACAO BRUTA'].rank(ascending=False, method='first').astype(int)
ranking_list = list(UNB_sorted[['NOME', 'DEPARTAMENTO', 'REMUNERACAO BRUTA', 'REMUNERACAO LIQUIDA', 'Ranking']].itertuples(index=False, name=None))
ranking_list[0]
with open("ranking_UNB.txt", "w") as f:
    for professor in ranking_list:
        nome = professor[0]
        departamento = professor[1]
        rem_bruta = format_brl(professor[2])
        rem_liq = format_brl(professor[3])
        num_lugar = professor[4]
        f.write("{nome} | {departamento} | REMUNERAÇÃO BRUTA DE {rem_bruta} | REMUNERAÇÃO LÍQUIDA DE {rem_liq} | {num_lugar} ({lugar}) LUGAR \n \n".format(departamento = departamento, num_lugar = num_lugar, nome = nome, lugar=Numero_ordinal(num_lugar), rem_bruta=rem_bruta, rem_liq = rem_liq))